In [6]:
import pdb 
import os 
import pickle 
import json 
import pandas as pd 
import numpy as np 
from scipy.stats import ttest_rel,ttest_ind, ttest_1samp 
import scipy
from matplotlib import pyplot as plt 
import seaborn as sns 
import math 
from sklearn.metrics import adjusted_rand_score 
from humanUtils import * 
from cswHumanDatabase import load_final_df, load_dfs 
from analysis import * 
from utils import * 
sns.set_context('talk')
plt.rcParams['font.size'] = 22
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load and save dataframes from each mturk experiemnt

In [12]:
ALL_CONDITIONS

['blocked',
 'interleaved',
 'blocked_rep',
 'interleaved_rep',
 'explicit_interleaved',
 'inserted_early',
 'inserted_middle',
 'inserted_late',
 'inserted_early_rep',
 'inserted_middle_rep',
 'inserted_late_rep']

In [13]:
## load and save
recompute = True
dfD = {}
for cond in ALL_CONDITIONS:
  print(cond)
  for thresh in [0.9,0]:
    fname = f"{cond}_thresh{int(thresh*100)}.csv"
    if recompute:
      df = load_final_df(cond,threshold=thresh) 
      df.to_csv(f'data/human/{fname}')
      
    df = pd.read_csv(f'data/human/{fname}')
    dfD[cond,thresh] = df
  print("________________")
human_df = pd.concat(dfD,names=['condition','thresh'])

blocked
./CSWparticipants.db

-loading code version: RT40B1000cl
N = 50 subjects
./CSWparticipants.db

-loading code version: RT40B1000cl
N = 50 subjects
________________
interleaved
./csw_mturk_spring19.db

-loading code version: RT01B1000cl
N = 55 subjects
./csw_mturk_spring19.db

-loading code version: RT01B1000cl
N = 55 subjects
________________
blocked_rep
./csw_mturk_spring19.db

-loading code version: csw1000block40.04.07.19
N = 63 subjects
./csw_mturk_spring19.db

-loading code version: csw1000block40.04.07.19
N = 63 subjects
________________
interleaved_rep
./csw_mturk_spring19.db

-loading code version: csw1000block01.04.25.19
N = 56 subjects
./csw_mturk_spring19.db

-loading code version: csw1000block01.04.25.19
N = 56 subjects
________________
explicit_interleaved
./csw_mturk_spring19.db

-loading code version: csw1000boldmdp.05.09.19
N = 52 subjects
./csw_mturk_spring19.db

-loading code version: csw1000boldmdp.05.09.19
N = 52 subjects
________________
inserted_early
./csw

## additional formating to human df

In [14]:
# additional columns
human_df.loc[:,'score'] = human_df.correct_response
human_df.loc[:,'response_node'] = human_df.apply(lambda r: [r.false_tonode,r.true_tonode][r.correct_response],axis=1)
node2stateD = {
  "BEGIN":0,
  "LOCNODEB":1,
  "LOCNODEC":2,
  "NODE11":3,
  "NODE12":4,
  "NODE21":5,
  "NODE22":6,
  "NODE31":7,
  "NODE32":8,
  "END":9
}
human_df.loc[:,'rfc_int'] = (human_df.true_rfc.str.split('-').str[-1].str.lower()=='jungle').astype('int')
human_df.loc[:,'response_node_int'] = human_df.apply(lambda r: node2stateD[r.response_node],axis=1)
# finalize
human_df = human_df.reset_index()
print(human_df.columns)
human_df.head()



Index(['condition', 'thresh', 'level_2', 'subjnum', 'block', 'story', 'depth',
       'Unnamed: 4', 'type', 'qtype', 'state', 'fromnode', 'true_tonode',
       'false_tonode', 'rfc', 'true_rfc', 'false_rfc', 'response_right',
       'true_on_right', 'correct_response', 'edge', 'score', 'response_node',
       'rfc_int', 'response_node_int'],
      dtype='object')


,condition,thresh,level_2,subjnum,block,story,depth,Unnamed: 4,type,qtype,...,true_rfc,false_rfc,response_right,true_on_right,correct_response,edge,score,response_node,rfc_int,response_node_int
0,blocked,0.9,0,6,0,0,1,11,question,transQ,...,subject-Beverly_location-jungle,subject-Beverly_location-jungle,True,False,0,NODE11NODE22,0,NODE21,1,5
1,blocked,0.9,1,6,0,0,2,15,question,transQ,...,subject-Beverly_location-jungle,subject-Beverly_location-jungle,True,False,0,NODE22NODE31,0,NODE32,1,8
2,blocked,0.9,2,6,0,1,1,30,question,transQ,...,subject-Jacob_location-jungle,subject-Jacob_location-jungle,True,True,1,NODE11NODE22,1,NODE22,1,6
3,blocked,0.9,3,6,0,1,2,34,question,transQ,...,subject-Jacob_location-jungle,subject-Jacob_location-jungle,True,True,1,NODE22NODE31,1,NODE31,1,7
4,blocked,0.9,4,6,0,2,1,49,question,transQ,...,subject-Denise_location-jungle,subject-Denise_location-jungle,True,True,1,NODE11NODE22,1,NODE22,1,6


In [15]:
human_df.level_2


0             0
1             1
2             2
3             3
4             4
          ...  
261781    16882
261782    16883
261783    16884
261784    16885
261785    16886
Name: level_2, Length: 261786, dtype: int64

condition: the curriculum type

thresh: this threshold is applied to the filler questions which test people's attentiveness to the task, so this has the effect of filtering out unattentive participants

level_2: 

subjnum: participant id

block

true_on_right: whether the forced choice option on the right is the true next state instead of left

response_right: whether or not participant chose the right instead of left option

qtype: transQ is the 2 alterntive forced choice asking about the next transition

type: if question, this means that this particular row is the response to a question

edge: combination of true_tonode and fromnode

story: which story of the 200 we give

true_rfc: correct role filler combination

false_rfc: false role filler combination (could be on right or left as indicated by true_on_right)

response_node: location of chosen role filler combination, NODEXY means node at depth X and Y = 1 for left and Y = 2 for right

# Now we have the data for the various mturk experiments we can run our own analyses on these below